In [1]:
import random

import numpy as np
import matplotlib.pyplot as plt

# Bokeh imports
from bokeh.io import output_notebook, show, save
from bokeh.plotting import figure, output_file, ColumnDataSource, reset_output
from bokeh.models import HoverTool
output_notebook()

from ipywidgets.widgets import Text

#scipy constants
from scipy.constants import Boltzmann as kB
from scipy.constants import Avogadro as NA
from scipy.constants import Planck as h
from scipy.constants import speed_of_light as c0
from scipy.constants import R

Loading BokehJS ...

### Set Global parameters

In [2]:
L_nm = 300  # length of the nanotube
R_nm = 1  # radius of the exiton
N_DEF = 10  # number of defects per nanotube
T_STEP_ps = 1 # time step
D_exc_nm_per_s = 1.07e15  # Exciton Diffusion Coefficient https://doi.org/10.1021/nn101612b

k_r_per_s = 1e10  # radiativ decay constant
k_nr_per_s = 1e11  # non-radiativ decay constant bright
k_d_per_s = 1e11  # constant for transion into dark state
k_n_per_s = 1e11


## Exciton Fate

Very small MC simulation without any dynamic elements. The exciton is allowed to radiative, non-ratidative decays as well as diffusion and falling into dark state (not in agreement with working modells) More for algorithm purposes

In [3]:
def exciton_fate():
    """Determines the fate of a single exciton.

    Returns
    -------
    exciton_fate : 1D array
        Matrix showing the binned fate of the MC steps. Order is
        k_r_per_s, k_nr_per_s, k_d_per_s, k_d_per_s
    """
    kin_const = [k_r_per_s, k_nr_per_s, k_d_per_s, k_d_per_s]
    fate = 3  # fate of single exciton in MC step
    exciton_fate = np.zeros(len(kin_const))

    while fate > 1:
        # calculate the probability for exciton decay
        p_fate = np.array([e * random.uniform(0, 1) for e in kin_const])

        # Store result for highest probability
        fate = p_fate.argmax()
        exciton_fate[fate] += 1

        # insurance that there won't be an endless loop
        if exciton_fate.sum() > 1e3:
            print('Simulation exceeds 1e3 steps, loop aborded')
            return exciton_fate
    return exciton_fate

In [4]:
def photons_fate(n_photons, func, func_kwargs={}):
    """Simulate the fate of numerous photons.
    For a key to the fate look a the respective exciton fate function.

    Parameters
    ----------
    n_photons : int
        Number of photons to be used in the simulation.
    func : callable
        Function to perform MC simulation of a single exciton.
    func_kwargs: dict
        Dictionary containing the arguments and keyword arguments for
        func.

    Returns
    -------
    photons_fate : 1D array
        Binnend fate of the photons in simulation.
    quantum_yield : 1D array
        Quantum yield of the E11 and E11* radiative decay."""
    info = Text()
    display(info)

    # initiate matrix size
    photons_fate = func(**func_kwargs)
    info.value = f"Processing photon (1/ {n_photons})"

    # loop for the desired number of photons
    for p in np.arange(n_photons-1):
        photons_fate += func(**func_kwargs)
        info.value = f"Processing photon ({p+2}/ {n_photons})"

    # calculate the quantum yield
    quantum_yield = photons_fate[:2] / n_photons

    return photons_fate, quantum_yield

In [5]:
photons_fate(1e3, exciton_fate)

Text(value='')

(array([   0., 1000.,  969.,  960.]), array([0., 1.]))

## Modell change

E11 can decay radiativ and non radiative, as well as transition to E11* (defect states) which can also decay radiative and non radiative. If the exciton gets trapped within E11* is determined with a 1D random walk

E11* transitions are marked with d for 'defect'

### Set Global Parameters

In [6]:
TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e11  # constant for radiativ decay from E11
k_d_r_per_s = 1e10  # constant for radiativ decay from E11*
k_nr_per_s = 1e11  # constant of non-radiativ decay from E11
k_d_nr_per_s = 1e11  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s) * TAU_d_ps / t_step

In [7]:
def create_defects(length=L_nm, n_def=N_DEF):
    """Creates defects along the CNT at random position.

    Parameters
    ----------
    length : int, optional
        Length of the CNT in nm, global constant as default.
    n_def : int, optional
        Number of defects on the CNT, global constant as default.

    Returns
    -------
    pos_def : 1D array
        Positions in nm of the defects on the CNT stored in
        array size (n_def, 1)
    """
    return np.random.randint(0, length, size=(n_def, 1))

In [8]:
def create_exciton(length=L_nm):
    """Creates exciton on the CNT at random position.

    Parameters
    ----------
    length : int, optional
        Length of the CNT in nm, global constant as default.

    Returns
    -------
    pos_exc : int
        Position of the exciton along the CNT as a random integer."""
    return random.randrange(length)

In [9]:
def exciton_walk(t_step, n_defects=10, length=L_nm):
    """
    
    Parameters
    ----------
    t_step : float
        Timestep in ps.
    n_defects : int, optional
        Number of defects on CNT. Default is 10
    length : int, optional
        Length of the CNT in nm, global constant as default.

    Returns
    -------
    exciton_fate : 1D array
        Array contains the binned fate of the exciton for each MC step:
        fate = 0 : E11 radiative decay
        fate = 1 : E11* radiative decay
        fate = 2 : E11 non-radiative decay
        fate = 3 : E11* non-radiative decay
        fate = 4 : Free diffusion walk
        fate = 5 : Exciton stays in trap
        fate = 6 : Exciton becomes trapped
    """

    kin_const = [k_r_per_s, k_d_r_per_s, k_nr_per_s, k_d_nr_per_s,
                 k_nothing(t_step), k_nothing_d(t_step)]

    # inital exciton is free in E11
    fate = 4
    trapped = 0

    # Initiate matrix to store exciton fate
    exciton_fate = np.zeros(len(kin_const)+1)

    # Inital position of the exciton and defects
    pos_exc_0 = create_exciton(length)
    defects = create_defects(length, n_defects)

    while fate > 3:

        # step if exciton is free
        if trapped == 0:
            pos_exc_1 = round(pos_exc_0 + (
                2 * D_exc_nm_per_s * t_step * 1e-12)**0.5)

        # quenching of the exciton at tube end
        if pos_exc_1 >= length:
            fate = 2
            exciton_fate[fate] += 1
            break

        # check if exciton became trapped
        if trapped == 0:
            pathway = np.arange(pos_exc_1-pos_exc_0)
            if np.in1d(pathway, defects).any():
                trapped = 1
                fate = 6
                exciton_fate[fate] += 1

        # fate of a trapped exciton
        if trapped == 1:
            # calculate probability for fate of trapped exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[1::2]])
            # Store result for highest probability
            fate = (2*p_fate.argmax() + 1)
            exciton_fate[fate] += 1

        # fate of freely diffusing exciton
        else:
            # calculate probability for fate of free exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[::2]])
            # Store result for highest probability
            fate = p_fate.argmax() * 2
            exciton_fate[fate] += 1

        # insurance that there won't be an endless loop
        if exciton_fate.sum() > 1e6:
            print('Simulation exceeds 1e6 steps, loop aborded')
            return exciton_fate

        # set position to new starting position
        pos_exc_0 = pos_exc_1

    return exciton_fate

In [10]:
photons_fate(5000, exciton_walk, {'t_step':1})

Text(value='')

(array([6.000000e+00, 1.800000e+01, 1.564000e+03, 3.412000e+03,
        2.518000e+03, 7.536765e+06, 3.430000e+03]), array([0.0012, 0.0036]))

## Modell addition
Add thermal detrapping

In [11]:
# termal detrapping 10.1021/acs.jpclett.8b03732
k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))

TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e11  # constant for radiativ decay from E11
k_d_r_per_s = 1e10  # constant for radiativ decay from E11*
k_nr_per_s = 1e11  # constant of non-radiativ decay from E11
k_d_nr_per_s = 1e11  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s + k_dt_per_s) * TAU_d_ps / t_step

In [12]:
def exciton_walk(t_step, n_defects=10, length=L_nm):
    """
    
    Parameters
    ----------
    t_step : float
        Timestep in ps.
    n_defects : int, optional
        Number of defects on CNT. Default is 10
    length : int, optional
        Length of the CNT in nm, global constant as default.

    Returns
    -------
    exciton_fate : 1D array
        Array contains the binned fate of the exciton for each MC step:
        fate = 0 : E11* radiative decay
        fate = 1 : E11 radiative decay
        fate = 2 : E11* non-radiative decay
        fate = 3 : E11 non-radiative decay
        fate = 4 : Exciton stays in trap
        fate = 5 : Free diffusion walk
        fate = 6 : Exciton escapes trap
        fate = 7 : Exciton becomes trapped
    """

    kin_const = [k_d_r_per_s, k_r_per_s, k_d_nr_per_s, k_nr_per_s,
                 k_nothing_d(t_step), k_nothing(t_step), k_dt_per_s]

    # inital exciton is free in E11
    fate = 4
    trapped = 0

    # Initiate matrix to store exciton fate
    exciton_fate = np.zeros(len(kin_const)+1)

    # Inital position of the exciton and defects
    pos_exc_0 = create_exciton(length)
    defects = create_defects(length, n_defects)

    while fate > 3:

        # step if exciton is free
        if trapped == 0:
            pos_exc_1 = round(pos_exc_0 + (
                2 * D_exc_nm_per_s * t_step * 1e-12)**0.5)

        # quenching of the exciton at tube end
        if pos_exc_1 >= length:
            fate = 3
            exciton_fate[fate] += 1
            break

        # check if exciton became trapped
        if trapped == 0:
            pathway = np.arange(pos_exc_1-pos_exc_0)
            if np.in1d(pathway, defects).any():
                trapped = 1
                fate = 7
                exciton_fate[fate] += 1

        # fate of a trapped exciton
        if trapped == 1:
            # calculate probability for fate of trapped exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[::2]])
            # Store result for highest probability
            fate = 2*p_fate.argmax()
            exciton_fate[fate] += 1

        # fate of freely diffusing exciton
        else:
            # calculate probability for fate of free exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[1::2]])
            # Store result for highest probability
            fate = (p_fate.argmax() * 2 + 1)
            exciton_fate[fate] += 1

        # insurance that there won't be an endless loop
        if exciton_fate.sum() > 1e6:
            print('Simulation exceeds 1e6 steps, loop aborded')
            return exciton_fate

        # set position to new starting position
        pos_exc_0 = pos_exc_1
        if fate ==7:
            pos_exc_0 += 1

    return exciton_fate

## Optimize rate constants to reflect Quantum Yield

In [13]:
# termal detrapping 10.1021/acs.jpclett.8b03732
k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))

TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e11  # constant for radiativ decay from E11
k_d_r_per_s = 1e10  # constant for radiativ decay from E11*
k_nr_per_s = 1e11  # constant of non-radiativ decay from E11
k_d_nr_per_s = 1e11  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s + k_dt_per_s) * TAU_d_ps / t_step
photons_fate(5000, exciton_walk, {'t_step':1})

Text(value='')

(array([2.500000e+01, 3.000000e+00, 3.386000e+03, 1.586000e+03,
        7.349639e+06, 2.545000e+03, 0.000000e+00, 3.411000e+03]),
 array([0.005 , 0.0006]))

In [15]:
#fate = 0 : E11 radiative decay
#fate = 1 : E11* radiative decay

In [16]:
# termal detrapping 10.1021/acs.jpclett.8b03732
k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))

TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e11  # constant for radiativ decay from E11
k_d_r_per_s = 1e9  # constant for radiativ decay from E11*
k_nr_per_s = 1e10  # constant of non-radiativ decay from E11
k_d_nr_per_s = 1e10  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s + k_dt_per_s) * TAU_d_ps / t_step
photons_fate(5000, exciton_walk, {'t_step':1})

Text(value='')

(array([1.200000e+01, 1.800000e+01, 3.446000e+03, 1.524000e+03,
        8.820354e+06, 2.508000e+03, 6.200000e+01, 3.458000e+03]),
 array([0.0024, 0.0036]))

In [17]:
# termal detrapping 10.1021/acs.jpclett.8b03732
k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))

TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e10  # constant for radiativ decay from E11
k_d_r_per_s = 1e09  # constant for radiativ decay from E11*
k_nr_per_s = 5e09  # constant of non-radiativ decay from E11
k_d_nr_per_s = 5e09  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s + k_dt_per_s) * TAU_d_ps / t_step
photons_fate(5000, exciton_walk, {'t_step':1})

Text(value='')

(array([3.1000000e+01, 1.2000000e+01, 3.4190000e+03, 1.5380000e+03,
        1.1054484e+07, 2.5320000e+03, 3.0000000e+02, 3.4500000e+03]),
 array([0.0062, 0.0024]))

In [24]:
# termal detrapping 10.1021/acs.jpclett.8b03732
k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))

TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e10  # constant for radiativ decay from E11
k_d_r_per_s = 5e09  # constant for radiativ decay from E11*
k_nr_per_s = 5e09  # constant of non-radiativ decay from E11
k_d_nr_per_s = 5e09  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s + k_dt_per_s) * TAU_d_ps / t_step
photons_fate(5000, exciton_walk, {'t_step':1})

Text(value='')

(array([1.7470000e+03, 7.0000000e+00, 1.6940000e+03, 1.5520000e+03,
        1.2142328e+07, 2.6940000e+03, 5.7000000e+01, 3.4410000e+03]),
 array([0.3494, 0.0014]))

In [27]:
# termal detrapping 10.1021/acs.jpclett.8b03732
k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))

TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e10  # constant for radiativ decay from E11
k_d_r_per_s = 2.5e09  # constant for radiativ decay from E11*
k_nr_per_s = 5e09  # constant of non-radiativ decay from E11
k_d_nr_per_s = 5e09  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s + k_dt_per_s) * TAU_d_ps / t_step
photons_fate(5000, exciton_walk, {'t_step':1})

Text(value='')

(array([4.5400000e+02, 7.0000000e+00, 3.0110000e+03, 1.5280000e+03,
        1.2369003e+07, 2.3180000e+03, 1.4100000e+02, 3.4650000e+03]),
 array([0.0908, 0.0014]))

In [18]:
def exciton_walk_fittable(t_step, kin_const, n_defects=10, length=L_nm):
    """
    
    Parameters
    ----------
    t_step : float
        Timestep in ps.
    n_defects : int, optional
        Number of defects on CNT. Default is 10
    length : int, optional
        Length of the CNT in nm, global constant as default.

    Returns
    -------
    exciton_fate : 1D array
        Array contains the binned fate of the exciton for each MC step:
        fate = 0 : E11* radiative decay
        fate = 1 : E11 radiative decay
        fate = 2 : E11* non-radiative decay
        fate = 3 : E11 non-radiative decay
        fate = 4 : Exciton stays in trap
        fate = 5 : Free diffusion walk
        fate = 6 : Exciton escapes trap
        fate = 7 : Exciton becomes trapped
    """

    # inital exciton is free in E11
    fate = 4
    trapped = 0

    # Initiate matrix to store exciton fate
    exciton_fate = np.zeros(len(kin_const)+1)

    # Inital position of the exciton and defects
    pos_exc_0 = create_exciton(length)
    defects = create_defects(length, n_defects)

    while fate > 3:

        # step if exciton is free
        if trapped == 0:
            pos_exc_1 = round(pos_exc_0 + (
                2 * D_exc_nm_per_s * t_step * 1e-12)**0.5)

        # quenching of the exciton at tube end
        if pos_exc_1 >= length:
            fate = 3
            exciton_fate[fate] += 1
            break

        # check if exciton became trapped
        if trapped == 0:
            pathway = np.arange(pos_exc_1-pos_exc_0)
            if np.in1d(pathway, defects).any():
                trapped = 1
                fate = 7
                exciton_fate[fate] += 1

        # fate of a trapped exciton
        if trapped == 1:
            # calculate probability for fate of trapped exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[::2]])
            # Store result for highest probability
            fate = 2*p_fate.argmax()
            exciton_fate[fate] += 1

        # fate of freely diffusing exciton
        else:
            # calculate probability for fate of free exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[1::2]])
            # Store result for highest probability
            fate = (p_fate.argmax() * 2 + 1)
            exciton_fate[fate] += 1

        # insurance that there won't be an endless loop
        if exciton_fate.sum() > 1e6:
            print('Simulation exceeds 1e6 steps, loop aborded')
            return exciton_fate

        # set position to new starting position
        pos_exc_0 = pos_exc_1
        if fate ==7:
            pos_exc_0 += 1

    return exciton_fate

In [34]:
def fit_func(x0):
    "x0 = k_d_r_per_s, k_r_per_s, k_d_nr_per_s, k_nr_per_s"
    tau_ps = 100
    tau_d_ps = 1000
    k_nothing = (x0[1] + x0[3]) * tau_ps
    k_nothing_d = (x0[0] + x0[2]) * tau_d_ps
    k_dt_per_s = 0.5 * (1e12/ 385 + 1e12/ 1132) + 0.1e12 * np.exp(-1.6182e-11 / (kB * 300))
    kin_const = list(x0)
    kin_const.append(k_nothing_d)
    kin_const.append(k_nothing)
    kin_const.append(k_dt_per_s)
    exciton_fate, QY = photons_fate(1000, exciton_walk_fittable, {'t_step':1, 'kin_const': kin_const})
    return (abs(np.array([0.1, 0.005])-QY).sum())

In [35]:
from scipy.optimize import minimize

In [36]:
x0 = [1e10, 2.5e09, 5e09, 5e09]

In [37]:
bounds = ((1e09, 1e11), (1e9, 1e10), (1e9, 1e10), (1e9, 1e10))

In [38]:
res = minimize(fit_func, x0, bounds=bounds)

Text(value='')

Text(value='')

Text(value='')

Text(value='')

Text(value='')

...

Text(value='')

Text(value='')

Text(value='')

Text(value='')

Text(value='')

In [39]:
res

      fun: 0.493
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([     0., 800000., 100000., 400000.])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 105
      nit: 0
   status: 2
  success: False
        x: array([1.0e+10, 2.5e+09, 5.0e+09, 5.0e+09])